In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import sys
sys.path.append('clustering/')

from utils import *

In [ ]:
import pickle
from sklearn.cluster import KMeans
d, w = 100, 10
K = 26
method = "kmeans"

start_year, end_year = 2010, 2021

with open(os.path.join(get_data_dir(), "2010s", "vectors.pkl"), 'rb') as f:
        vectors = pickle.load(f)
kmeans = KMeans(n_clusters=K, random_state=42).fit(vectors)

with open(os.path.join(get_data_dir(), "2010s", f'cluster_labels_{method}.pkl'), 'wb') as f:
        pickle.dump(kmeans.labels_,f)
with open(os.path.join(get_data_dir(), "2010s", f'cluster_centers_{method}.pkl'), 'wb') as f:
        pickle.dump(kmeans.cluster_centers_, f)

/home/lyuzhuoqi/miniconda3/envs/p2v/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [ ]:
import pandas as pd

cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", "cluster_df.parquet"))
cluster_df

data_dir:  /home/lyuzhuoqi/projects/clustering/data


,Scopus_label,movMF_label,movMF_distance,x_val,y_val,kmeans_label,closest_center_distance
VID,,,,,,,
202381698,Multidisciplinary,22,0.445886,-67.928200,15.572327,17,0.628846
137773608,Multidisciplinary,22,0.590942,-68.405334,-55.633186,17,0.735654
125754415,Multidisciplinary,22,0.574571,-68.448853,-55.613579,17,0.705024
3880285,Multidisciplinary,23,0.618842,-68.407288,-55.634430,8,0.724859
111155417,Chemistry,23,0.220853,-54.506985,-61.217068,11,0.495787
...,...,...,...,...,...,...,...
2764485818,Medicine,21,0.268070,-28.969574,34.819569,18,0.542531
83454320,Arts and Humanities,25,0.034777,78.609909,31.736822,13,0.251599
16507453,Arts and Humanities,6,0.113656,89.206772,17.625090,13,0.307089


In [73]:
# 生成一个字典，key为kmeans_label，value为该kmeans_label中出现次数最多的Scopus_label
label_counts = cluster_df.groupby(['kmeans_label', 'Scopus_label']).size().unstack(fill_value=0)
kmeans2Scopus = label_counts.idxmax(axis=1).to_dict()

kmeans2Scopus

{0: 'Social Sciences',
 1: 'Social Sciences',
 2: 'Computer Science',
 3: 'Medicine',
 4: 'Medicine',
 5: 'Engineering',
 6: 'Physics and Astronomy',
 7: 'Medicine',
 8: 'Agricultural and Biological Sciences',
 9: 'Psychology',
 10: 'Engineering',
 11: 'Chemistry',
 12: 'Earth and Planetary Sciences',
 13: 'Arts and Humanities',
 14: 'Engineering',
 15: 'Mathematics',
 16: 'Veterinary',
 17: 'Biochemistry, Genetics and Molecular Biology',
 18: 'Medicine',
 19: 'Social Sciences',
 20: 'Medicine',
 21: 'Agricultural and Biological Sciences',
 22: 'Medicine',
 23: 'Social Sciences',
 24: 'Business, Management and Accounting',
 25: 'Social Sciences'}

In [ ]:
import pandas as pd
import os
import plotly.graph_objects as go
from matplotlib.colors import LinearSegmentedColormap, to_rgba

# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", "cluster_df.parquet"))

# 获取唯一标签
scopus_labels = cluster_df['Scopus_label'].unique()
kmeans_labels = cluster_df['kmeans_label'].unique()

# 生成颜色
Scopus_colors = [disc2color[scopus_label] for scopus_label in scopus_labels]
kmeans_colors = [disc2color[kmeans2Scopus[kmeans_label]] for kmeans_label in kmeans_labels]

# 创建标签映射
label_map = {label: i for i, label in enumerate(scopus_labels)}
label_map.update({label: i + len(scopus_labels) for i, label in enumerate(kmeans_labels)})

# 创建桑基图数据
cluster_group = cluster_df.groupby(['Scopus_label', 'kmeans_label']).size().reset_index(name='count')
source = cluster_group['Scopus_label'].map(label_map).tolist()
target = cluster_group['kmeans_label'].map(label_map).tolist()
value = cluster_group['count'].tolist()

# 创建渐变颜色，并降低不透明度
link_colors = []
alpha = 0.5  # 设置透明度，范围 [0, 1]
for i, row in cluster_group.iterrows():
    source_color = to_rgba(disc2color[row['Scopus_label']], alpha=1.0)
    target_color = to_rgba(disc2color[kmeans2Scopus[row['kmeans_label']]], alpha=1.0)
    cmap = LinearSegmentedColormap.from_list("source_target", [source_color, target_color])
    mid_color = cmap(0.5)  # 在中间点取颜色
    link_colors.append(f'rgba({mid_color[0]*255},{mid_color[1]*255},{mid_color[2]*255},{alpha})')  # 设置 alpha

# 创建桑基图
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=10,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=list(scopus_labels) + list(kmeans_labels),
        color=Scopus_colors + kmeans_colors
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=link_colors  # 设置渐变颜色
    )
)])

# 更新布局
fig.update_layout(title_text="Scopus_label to kmeans_label", font_size=10,
                  autosize=False, width=800, height=800)

# 显示图表
fig.show()

data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [137]:
# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", "cluster_df.parquet"))

# 统计流量信息，按 Scopus_label 和 kmeans_label 分组计数
flow_data = (
    cluster_df.groupby(["Scopus_label", "kmeans_label"])
    .size()
    .reset_index(name="flow")
    .apply(lambda row: [row["Scopus_label"], str(row["kmeans_label"]), row["flow"]], axis=1)
    .tolist()
)
flow_data

data_dir:  /home/lyuzhuoqi/projects/clustering/data


[['Agricultural and Biological Sciences', '1', 37],
 ['Agricultural and Biological Sciences', '2', 1],
 ['Agricultural and Biological Sciences', '5', 2],
 ['Agricultural and Biological Sciences', '8', 616],
 ['Agricultural and Biological Sciences', '9', 2],
 ['Agricultural and Biological Sciences', '11', 7],
 ['Agricultural and Biological Sciences', '12', 37],
 ['Agricultural and Biological Sciences', '13', 2],
 ['Agricultural and Biological Sciences', '14', 11],
 ['Agricultural and Biological Sciences', '15', 3],
 ['Agricultural and Biological Sciences', '16', 104],
 ['Agricultural and Biological Sciences', '17', 12],
 ['Agricultural and Biological Sciences', '18', 1],
 ['Agricultural and Biological Sciences', '21', 526],
 ['Agricultural and Biological Sciences', '22', 1],
 ['Agricultural and Biological Sciences', '24', 8],
 ['Agricultural and Biological Sciences', '25', 1],
 ['Arts and Humanities', '0', 54],
 ['Arts and Humanities', '1', 14],
 ['Arts and Humanities', '2', 9],
 ['Arts

In [107]:
node_appearance_order = []
for entry in flow_data:
    if entry[0] not in node_appearance_order:
        node_appearance_order.append(entry[0])
    if int(entry[1]) not in node_appearance_order:
        node_appearance_order.append(int(entry[1]))
node_appearance_order

['Agricultural and Biological Sciences',
 1,
 2,
 5,
 8,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 21,
 22,
 24,
 25,
 'Arts and Humanities',
 0,
 3,
 6,
 7,
 19,
 20,
 23,
 'Biochemistry, Genetics and Molecular Biology',
 4,
 10,
 'Business, Management and Accounting',
 'Chemical Engineering',
 'Chemistry',
 'Computer Science',
 'Decision Sciences',
 'Dentistry',
 'Earth and Planetary Sciences',
 'Economics, Econometrics and Finance',
 'Energy',
 'Engineering',
 'Environmental Science',
 'Health Professions',
 'Immunology and Microbiology',
 'Materials Science',
 'Mathematics',
 'Medicine',
 'Multidisciplinary',
 'Neuroscience',
 'Nursing',
 'Pharmacology, Toxicology and Pharmaceutics',
 'Physics and Astronomy',
 'Psychology',
 'Social Sciences',
 'Veterinary']

In [108]:
for i, label in enumerate(node_appearance_order):
    if type(label) == str:
        node_appearance_order[i] = disc2color[label]
    else:
        node_appearance_order[i] = disc2color[kmeans2Scopus[label]]

In [109]:
node_appearance_order

['#000099',
 '#66FF66',
 '#FF5C29',
 '#D20000',
 '#000099',
 '#92D050',
 '#9A0000',
 '#FE0000',
 '#375623',
 '#D20000',
 '#FBFF57',
 '#CC00FF',
 '#336699',
 '#7030A0',
 '#000099',
 '#7030A0',
 '#187402',
 '#66FF66',
 '#375623',
 '#66FF66',
 '#7030A0',
 '#FFCC00',
 '#7030A0',
 '#66FF66',
 '#7030A0',
 '#66FF66',
 '#336699',
 '#7030A0',
 '#D20000',
 '#187402',
 '#6C0000',
 '#9A0000',
 '#FF5C29',
 '#16A90F',
 '#A679FF',
 '#FE0000',
 '#8FA329',
 '#FF7C80',
 '#D20000',
 '#D26B04',
 '#CCB3FF',
 '#0000F2',
 '#FC9320',
 '#FBFF57',
 '#7030A0',
 '#000000',
 '#0099FF',
 '#9900CC',
 '#85D6FF',
 '#FFCC00',
 '#92D050',
 '#66FF66',
 '#CC00FF']

In [162]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# 自定义词映射
word_mapping = {'mechanics': 'mechanic',
                'mechanical': 'mechanic',
                'electrical': 'electric',
                'electronics': 'electric',
                'financial': 'finance',
                'political': 'politics',
                'historical': 'history',
                'computer': 'computing',
                'intelligent': 'intelligence',
                'agricultural': 'agriculture',
                'educational': 'education',
                'dental': 'dentistry',
                'archaeological': 'archaeology',
                'mathematical': 'mathematics',
                'mathematica': 'mathematics',
                'matematico': 'mathematics',
                'mathematicae': 'mathematics',
                'economic': 'economics',
                'chemical': 'chemistry',
                'geophysical': 'geophysics',
                'botanical': 'botany',
                'physical': 'physics',
                'entomological': 'entomology', 
                'entomologist': 'entomology',
                'biological': 'biology',
                'geographical': 'geography',
                'geological': 'geology',
                'geographer': 'geography',
                'cells': 'cell',
                'policy': 'politics',
                'psychological': 'psychology',
                'linguistical': 'linguistics',
                'linguistica': 'linguistics',
                'medical': 'medicine',
}
# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", "cluster_df.parquet"))
cluster_df = cluster_df.merge(pd.read_parquet(os.path.join(get_data_dir(), "MAG_venue.parquet")), left_index=True, right_index=True)

# 定义预处理函数，进行自定义词映射
def preprocess_text(text):
    words = text.lower().split()  # 将文本小写并按空格分词
    mapped_words = [word_mapping[word] if word in word_mapping else word for word in words]
    filtered_words = [word for word in mapped_words]
    return ' '.join(filtered_words)

# 按 cluster_label 分组，将 OriginalVenue 文本合并为一个文档，并进行预处理
cluster_docs = (
    cluster_df.groupby('kmeans_label').parallel_apply(lambda x: preprocess_text(' '.join(x.OriginalVenue))).tolist()
)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [163]:
# 自定义停用词
custom_stop_words = ['results', 'model', 'based', 'data', 'proposed', 'study', 'paper', 'results', 
                     'system', 'patients', 'using', 'method', 'health', 'two', 'article', 'la', 'que', 'en',
                     'el', 'et', 'un', 'one', 'also', 'high', 'properties', 'methods', 'among', 'new',
                     'sp', 'use', 'group', 'used', 'process', 'kg', 'abstract', 'different', 'time',
                     'treatment', 'analysis', 'different', 'performance', 'le', 'los', 'se', 'les',
                     'mm', 'may', 'de', 'del', 'des', 'nov', 'found', 'research', 'showed', 'las',
                     'development', 'years', 'da', 'studies', 'first', 'findings', 'di', 'however', 'three',
                     'associated', 'relationship', 'design', 'em', 'approach', 'risk', 'patient', 'care',
                     '95', 'age', 'ci', 'compared', '10', 'background', 'conclusions', 'significant',
                     'higher', 'vs', 'mortality', 'therapy', 'increased', 'significantly', '19', 'children',
                     'find', 'show', 'problem', 'mathrm', 'included', 'mean', 'systems', '12', 'effects',
                     'participants', 'related', 'cases', 'disease', 'levels', 'outcomes', 'non', 'total',
                     'factors', 'conclusion', 'nm', 'ra', 'year', 'dr', 'journal', 'review', 'international',
                     'science', 'chinese', 'university'
                     ]

# 计算 TF-IDF
top_words_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english')+custom_stop_words)
tfidf_matrix = top_words_vectorizer.fit_transform(cluster_docs)

In [164]:
top_n = 5  # 每个聚类提取前 n 个关键词
# 获取每个聚类中最重要的关键词
top_words = {}
feature_names = top_words_vectorizer.get_feature_names_out()

# 确保 cluster_label 的顺序与 cluster_docs 一致
cluster_labels = sorted(cluster_df['kmeans_label'].unique())

for j, cluster_label in enumerate(cluster_labels):
    tfidf_scores = tfidf_matrix[j].toarray().flatten()
    top_indices = tfidf_scores.argsort()[-top_n:][::-1]  # 获取前 n 个关键词的索引
    top_words[cluster_label] = [feature_names[idx] for idx in top_indices]

top_words

{0: ['law', 'politics', 'economics', 'european', 'public'],
 1: ['geography', 'environmental', 'economics', 'politics', 'planning'],
 2: ['computing', 'information', 'acm', 'transactions', 'ieee'],
 3: ['dentistry', 'oral', 'surgery', 'ophthalmology', 'dermatology'],
 4: ['oncology', 'cancer', 'surgery', 'clinical', 'gastroenterology'],
 5: ['ieee', 'electric', 'engineering', 'transactions', 'computing'],
 6: ['physics', 'ieee', 'electric', 'materials', 'optics'],
 7: ['medicine', 'clinical', 'cardiovascular', 'cardiology', 'heart'],
 8: ['ecology', 'entomology', 'biology', 'zoology', 'conservation'],
 9: ['psychology', 'psychiatry', 'mental', 'social', 'child'],
 10: ['technology', 'engineering', 'information', 'optics', 'laser'],
 11: ['chemistry', 'materials', 'polymer', 'engineering', 'catalysis'],
 12: ['geology', 'earth', 'geophysics', 'water', 'engineering'],
 13: ['history', 'archaeology', 'literature', 'philosophy', 'historia'],
 14: ['engineering', 'mechanic', 'materials', 't

In [165]:
# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", "cluster_df.parquet"))

# 统计流量信息，按 Scopus_label 和 kmeans_label 分组计数
flow_data = (
    cluster_df.groupby(["Scopus_label", "kmeans_label"])
    .size()
    .reset_index(name="flow")
    .apply(lambda row: [row["Scopus_label"], str(row["kmeans_label"]), row["flow"]], axis=1)
    .tolist()
)

for entry in flow_data:
    entry[1] = ','.join(top_words[int(entry[1])])

data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [166]:
flow_data

[['Agricultural and Biological Sciences',
  'geography,environmental,economics,politics,planning',
  37],
 ['Agricultural and Biological Sciences',
  'computing,information,acm,transactions,ieee',
  1],
 ['Agricultural and Biological Sciences',
  'ieee,electric,engineering,transactions,computing',
  2],
 ['Agricultural and Biological Sciences',
  'ecology,entomology,biology,zoology,conservation',
  616],
 ['Agricultural and Biological Sciences',
  'psychology,psychiatry,mental,social,child',
  2],
 ['Agricultural and Biological Sciences',
  'chemistry,materials,polymer,engineering,catalysis',
  7],
 ['Agricultural and Biological Sciences',
  'geology,earth,geophysics,water,engineering',
  37],
 ['Agricultural and Biological Sciences',
  'history,archaeology,literature,philosophy,historia',
  2],
 ['Agricultural and Biological Sciences',
  'engineering,mechanic,materials,technology,structures',
  11],
 ['Agricultural and Biological Sciences',
  'mathematics,applications,geometry,algebra